# Model With GPU (Fine tuning the model using LoRA)

In [2]:
from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer, GenerationConfig
import torch
from peft import LoraConfig, TaskType, get_peft_model, PeftModel, PeftConfig
from tqdm import tqdm

## Import the data from Hugging face

In [ ]:
# dataset = load_dataset('samhog/psychology-10k', split = ['train'])[0]
# dataset.save_to_disk('./dataset')

dataset = load_from_disk('./dataset')
dataset

let's walk trough the data

In [ ]:
def print_conv(me, model):
    for i, j in zip(me, model):
        print(
            f"""
Me : {i}

Param mitr : {j}\n\n
            """
        )

In [ ]:
print_conv(dataset['input'][:3], dataset['output'][:3])

In [ ]:
dataset.shape

In [ ]:
Dataset = dataset.train_test_split(test_size = 0.2)
Dataset

## Import our model with its tokenizer

In [ ]:
# if you want to use QLoRA use this but remember in 
# normal inference you have to use GPU

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_storage=torch.bfloat16,
# )

In [ ]:
# Importing model from the Hugging Face hub

model_name = 'google/flan-t5-small'
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype = torch.bfloat16,
    # quantization_config = bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Saving the model

# model.save_pretrained('/kaggle/working/Model')
# tokenizer.save_pretrained('/kaggle/working/Model')

Let's try some stuff with tokenizer

In [ ]:
tokenizer("hey", return_token_type_ids = False)

In [ ]:
device = torch.device('cuda')
tokenizer.decode(
    model.generate(
        tokenizer(
            dataset['input'][0], return_token_type_ids = False, return_tensors = 'pt', padding=True, truncation=True
        )['input_ids'].to(device)
    )[0], skip_special_tokens= True
)

In [ ]:
def inference(input_data, param_mitr = model):
    intruct = dataset['instruction'][0]
    task = "Answer :"
    inp = [intruct + "\n" + sent + task for sent in input_data]
    output = param_mitr.generate(
        tokenizer(
            inp, return_token_type_ids = False, return_tensors = 'pt', padding=True, truncation=True
        )['input_ids'].to(device),
        generation_config = GenerationConfig(max_new_token = 200)
    )
    decoded = [tokenizer.decode(out, skip_special_tokens= True) for out in output]
    print_conv(input_data, decoded)

In [ ]:
inference(dataset['input'][:4])

The model is showing good result on ICL (In Context Learning) with zero shot inference.

In [ ]:
def tokenize_function(example):
    intruct = dataset['instruction'][0]
    task = "Answer :"
    inp = [intruct + "\n" + question + '\n' + task for question in example['input']]
    example['input_ids'] = tokenizer(inp, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example['output'], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example


tokenized_datasets = Dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['input', 'output', 'instruction'])

## Fine tune our model (Using LoRA)

In [ ]:
config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r = 32,
    target_modules = "all-linear",
    lora_alpha=32,
    lora_dropout=0.05
)

In [ ]:
Model = get_peft_model(model, config)
Model.print_trainable_parameters()

In [ ]:
TrainArgs = TrainingArguments(
    output_dir = './Model',
    learning_rate = 1e-3,
    num_train_epochs = 2,
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4,
    weight_decay = 0.01,
    eval_strategy = "epoch",
    save_strategy = "epoch",
    logging_steps=1,
    load_best_model_at_end = True,
    report_to = ['tensorboard']
)

In [ ]:
trainer = Trainer(
    model = Model,
    args = TrainArgs,
    train_dataset = tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['test']
)

In [ ]:
import gc
# To clear the GPU
def clean():
    for i in range(15):
      gc.collect()
      torch.cuda.empty_cache()

In [ ]:
clean()
trainer.train()

In [ ]:
def peft_infrence_with_GPU(input_data, model = Model):
    intruct = dataset['instruction'][0]
    task = "Answer :"
    inp = [intruct + "\n" + sent +'\n'+ task for sent in input_data]
    output = model.generate(
        **tokenizer(
            inp, return_token_type_ids = False, return_tensors = 'pt', padding=True, truncation=True
        ).to('cuda'),
        generation_config = GenerationConfig(max_new_tokens=100, num_beams=1)
    )
    decoded = [tokenizer.decode(out, skip_special_tokens= True) for out in output]
    print_conv(input_data, decoded)

In [ ]:
peft_infrence_withGPU(Dataset['test']['input'][:4])

In [ ]:
trainer.model.save_pretrained('./peft_for_param_mitr')

# Model without GPU (using saved fine-tuned model)

## Load model from local storage

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('/kaggle/working/Model', device_map="auto")
model = AutoModelForSeq2SeqLM.from_pretrained('/kaggle/working/Model',
                                              device_map="auto",
                                              torch_dtype = torch.bfloat16
                                             )

## Load PEFT model 

In [ ]:
peft_model_path = './peft_for_param_mitr'
Model = PeftModel.from_pretrained(
            model,
            peft_model_path,
            is_trainable = False
            )

In [ ]:
def peft_infrence(input_data, model = Model):
    intruct = dataset['instruction'][0]
    task = "Answer :"
    inp = [intruct + "\n" + sent +'\n'+ task for sent in input_data]
    output = model.generate(
        **tokenizer(
            inp, return_token_type_ids = False, return_tensors = 'pt', padding=True, truncation=True
        ),
        generation_config = GenerationConfig(max_new_tokens=200, num_beams=1)
    )
    decoded = [tokenizer.decode(out, skip_special_tokens= True) for out in output]
    print_conv(input_data, decoded)

In [ ]:
peft_infrence(Dataset['test']['input'][:4])